You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 
Title: Unveiling the Future: A Deep Dive into Artificial Intelligence

Outline:
I. Introduction
- Definition of Artificial Intelligence
- Importance of AI in today's technological landscape
- Brief overview of key players and recent trends in AI

II. Latest Trends in Artificial Intelligence
- Advancements in deep learning and neural networks
- Growth of AI-powered personal assistants
- Impact of AI on various industries such as healthcare, finance, and marketing

III. Key Players in the AI Indu

I now can give a great answer

Final Answer:
# Unveiling the Future: A Deep Dive into Artificial Intelligence

**Introduction**

Artificial Intelligence, or AI, is the emulation of human intelligence processes by machines, specifically computer systems. In today's fast-paced technological landscape, AI is a driving force behind innovation and industry transformation. Major players such as Google, Amazon, and IBM are leading the way in AI research, constantly pushing boundaries and expanding possibilities.

**Latest Trends in Artificial Intelligence**

Advancements in deep learning and neural networks have vastly improved the capabilities of AI systems, allowing them to perform complex tasks with exceptional accuracy. The integration of AI-powered personal assistants like Siri and Alexa has streamlined everyday tasks, making them more convenient and efficient. In sectors like healthcare, finance, and marketing, AI is being utilized to optimize processes, analyze data, and make predictio

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Future: A Deep Dive into Artificial Intelligence

**Introduction**

Artificial Intelligence, or AI, is the emulation of human intelligence processes by machines, specifically computer systems. In today's fast-paced technological landscape, AI is a driving force behind innovation and industry transformation. Major players such as Google, Amazon, and IBM are leading the way in AI research, constantly pushing boundaries and expanding possibilities.

**Latest Trends in Artificial Intelligence**

Advancements in deep learning and neural networks have vastly improved the capabilities of AI systems, allowing them to perform complex tasks with exceptional accuracy. The integration of AI-powered personal assistants like Siri and Alexa has streamlined everyday tasks, making them more convenient and efficient. In sectors like healthcare, finance, and marketing, AI is being utilized to optimize processes, analyze data, and make predictions that were once considered unattainable.

**Key Players in the AI Industry**

Prominent companies are heavily investing in AI research and development to maintain a competitive edge. Google, known for its deep learning algorithms, is transforming search engines and data analysis. Amazon's AI-driven recommendation system is reshaping the e-commerce landscape. IBM's Watson platform is a trailblazer in natural language processing and cognitive computing. Keeping abreast of the latest news and advancements in the AI industry is crucial for tech professionals, business owners seeking to adopt AI solutions, and students/researchers fascinated by the future of AI technology.

**Target Audience Analysis**

This blog post provides valuable insights for tech professionals aiming to stay abreast of AI trends. Business owners interested in implementing AI solutions in their companies can gain a deeper understanding of AI's potential impact on operations. Students and researchers curious about the future of AI technology will discover the latest industry trends and breakthroughs.

In conclusion, Artificial Intelligence is a dynamic field with boundless opportunities. By remaining informed and actively engaging with the latest trends and advancements, individuals and organizations can leverage AI's power to drive innovation and achieve success.

*Stay ahead of the curve by subscribing to our newsletter for monthly updates on the latest trends and developments in Artificial Intelligence.*

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Quantum Technology"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Quantum Technology.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Title: Unleashing the Potential of Quantum Technology: A Comprehensive Guide

Introduction:
- Brief overview of quantum technology and its significance in the current technological landscape
- Mention of key players and recent advancements in the field

Key Points:
1. Latest Trends in Quantum Technology:
   - Quantum computing: advancements in qubit technology, quantum supremacy
   - Quantum communication: quantum key distribution, quantum internet
   - Quantum sensing and imaging: applications in healthcare, secu

I now can give a great answer

Final Answer:

# Unleashing the Potential of Quantum Technology: A Comprehensive Guide

In today's rapidly advancing technological landscape, quantum technology has emerged as a groundbreaking field with the potential to revolutionize various industries. Quantum technology harnesses the principles of quantum mechanics to develop innovative solutions that go beyond the capabilities of classical technology. With key players such as IBM, Google, and Microsoft leading the way in research and development, recent advancements in quantum computing, communication, sensing, and imaging have propelled this field to the forefront of scientific innovation.

## Latest Trends in Quantum Technology:

### Quantum Computing:
One of the most significant trends in quantum technology is the rapid progress in quantum computing. Advancements in qubit technology have enabled researchers to achieve quantum supremacy, where quantum computers outperform classical supercomputers in

In [14]:
Markdown(result)

# Unleashing the Potential of Quantum Technology: A Comprehensive Guide

In today's rapidly advancing technological landscape, quantum technology has emerged as a groundbreaking field with the potential to revolutionize various industries. Quantum technology harnesses the principles of quantum mechanics to develop innovative solutions that go beyond the capabilities of classical technology. With key players such as IBM, Google, and Microsoft leading the way in research and development, recent advancements in quantum computing, communication, sensing, and imaging have propelled this field to the forefront of scientific innovation.

## Latest Trends in Quantum Technology:

### Quantum Computing:
One of the most significant trends in quantum technology is the rapid progress in quantum computing. Advancements in qubit technology have enabled researchers to achieve quantum supremacy, where quantum computers outperform classical supercomputers in certain tasks. Quantum computing holds the promise of solving complex problems in areas such as cryptography, optimization, and material science that are currently infeasible with classical computers.

### Quantum Communication:
Quantum communication is another key trend in the field, with developments in quantum key distribution and the concept of a quantum internet. These technologies offer secure communication channels that are immune to eavesdropping, making them crucial for safeguarding sensitive information in various sectors, including finance, defense, and healthcare.

### Quantum Sensing and Imaging:
In healthcare, security, and environmental monitoring, quantum sensing and imaging technologies are making significant strides. Quantum sensors have the potential to revolutionize medical diagnostics, enhance security systems, and improve environmental monitoring processes by providing ultra-sensitive measurements with high precision and accuracy.

## Key Players in Quantum Technology:

Leading the charge in quantum computing research are industry giants such as IBM, Google, Microsoft, and Intel. These companies are investing heavily in developing quantum hardware and software solutions to unlock the full potential of quantum computing for real-world applications. Additionally, pioneers in quantum communication technologies like ID Quantique and Quintessence Labs are making significant contributions to secure communication infrastructures worldwide. In the quantum computing industry, notable players such as Rigetti Computing and D-Wave Systems are pushing the boundaries of quantum technology with their innovative approaches and solutions.

## Noteworthy News in Quantum Technology:

Recent breakthroughs in quantum error correction codes have addressed one of the major challenges in quantum computing, paving the way for more reliable and stable quantum systems. Collaborations between tech giants and research institutions are driving innovation in quantum technology, leading to the development of novel applications and solutions. Furthermore, government investments and initiatives aimed at promoting quantum research and development are fueling the growth of the quantum technology industry and fostering a vibrant ecosystem of innovation.

In conclusion, quantum technology holds immense potential to transform industries and drive scientific progress in the coming years. By staying informed about the latest trends, key players, and notable developments in quantum technology, technology enthusiasts, researchers, and industry professionals can gain valuable insights and stay ahead of the curve in this exciting field of innovation.

This blog post aims to provide a comprehensive guide to quantum technology, covering the latest trends, key players, and noteworthy news in the field. By exploring the implications of quantum technology on various industries and encouraging readers to delve deeper into this transformative technology, this article seeks to educate and inspire its audience to embrace the future of quantum technology.

Would you like to learn more about the latest trends in quantum technology and its impact on industries? Explore further resources and stay informed about the cutting-edge advancements in this rapidly evolving field.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).